# Neo4j

## Import pacchetti

In [1]:
# Installazione pacchetti
%pip install regex

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import pacchetti
import pandas as pd
import numpy as np
from neo4j import GraphDatabase
import regex as re

## Nodes

### `POKEMON nodes`

#### Load and Processing

In [34]:
# Load csv
pok=pd.read_csv('C:/Users/marco/OneDrive/UNIMIB_DataScience/99-PROJECT/DataManagement2022_Project/Dataset integrati 2.0/df_pokemon.csv', sep=',')
pok.head()

,#,Name,Species,Variant,Generation,Rarity,Evolves_from,Has_gender_diff,Type1,Type2,...,Attack,Defense,Sp. Atk,Sp. Def,Speed,image_url,VGC2022_rules,Monthly Usage (k),Usage Percent (%),Monthly Rank
0,1,Bulbasaur,Bulbasaur,NaN,1,Common,NaN,False,Grass,Poison,...,49.0,49.0,65.0,65.0,45.0,https://img.pokemondb.net/sprites/sword-shield...,Permitted,NaN,NaN,NaN
1,2,Ivysaur,Ivysaur,NaN,1,Common,Bulbasaur,False,Grass,Poison,...,62.0,63.0,80.0,80.0,60.0,https://img.pokemondb.net/sprites/sword-shield...,Permitted,NaN,NaN,NaN
2,3,Venusaur,Venusaur,NaN,1,Common,Ivysaur,True,Grass,Poison,...,82.0,83.0,100.0,100.0,80.0,https://img.pokemondb.net/sprites/sword-shield...,Permitted,204,7,24
3,3,Mega Venusaur,Venusaur,Mega,6,Common,Venusaur,True,Grass,Poison,...,100.0,123.0,122.0,120.0,80.0,https://img.pokemondb.net/sprites/sword-shield...,Banned,NaN,NaN,NaN
4,3,Venusaur Gigantamax,Venusaur,Gigantamax,8,NaN,Venusaur,NaN,Grass,Poison,...,NaN,NaN,NaN,NaN,NaN,https://archives.bulbagarden.net/media/upload/...,Gigantamax Allowed,NaN,NaN,NaN


In [35]:
# Mi adeugo alle convenzioni di Neo4j per i nomi delle label
pok=pok.rename(columns={'#':'NationalPokedexNumber','Sp. Atk':'SpAtk', 'Sp. Def':'SpDef', 'VGC2022_rules':'Vgc2022Rules', 'Monthly Usage (k)':'MonthlyUsageMarch2022', 'Usage Percent (%)':'UsagePercentMarch2022', 'Monthly Rank':'MonthlyRankMarch2022'})
pok = pok.drop(['Evolves_from', 'Has_gender_diff'], axis=1) # Non sono proprietà del nodo pokemon, tolgo
pok=pok.replace(np.nan, '',regex=True) # Tolgo i valori Nan

In [36]:
pok.head()

,NationalPokedexNumber,Name,Species,Variant,Generation,Rarity,Type1,Type2,Total,HP,Attack,Defense,SpAtk,SpDef,Speed,image_url,Vgc2022Rules,MonthlyUsageMarch2022,UsagePercentMarch2022,MonthlyRankMarch2022
0,1,Bulbasaur,Bulbasaur,,1,Common,Grass,Poison,318.0,45.0,49.0,49.0,65.0,65.0,45.0,https://img.pokemondb.net/sprites/sword-shield...,Permitted,,,
1,2,Ivysaur,Ivysaur,,1,Common,Grass,Poison,405.0,60.0,62.0,63.0,80.0,80.0,60.0,https://img.pokemondb.net/sprites/sword-shield...,Permitted,,,
2,3,Venusaur,Venusaur,,1,Common,Grass,Poison,525.0,80.0,82.0,83.0,100.0,100.0,80.0,https://img.pokemondb.net/sprites/sword-shield...,Permitted,204,7,24
3,3,Mega Venusaur,Venusaur,Mega,6,Common,Grass,Poison,625.0,80.0,100.0,123.0,122.0,120.0,80.0,https://img.pokemondb.net/sprites/sword-shield...,Banned,,,
4,3,Venusaur Gigantamax,Venusaur,Gigantamax,8,,Grass,Poison,,,,,,,,https://archives.bulbagarden.net/media/upload/...,Gigantamax Allowed,,,


In [ ]:
pok.to_csv('df_pokemon_import.csv')

#### Creazione delle query

##### Vincoli e indici

In [37]:

pok_constraint = '''
CREATE CONSTRAINT pokemonNameKey IF NOT EXISTS
FOR (p:Pokemon)
REQUIRE p.Name IS UNIQUE
'''

##### Nodi

In [38]:
# Creo i nodi e per ogni proprietà che aggiungo vado a controllare se nel file il dato è presente: se si, aggiungo la proprietà e inserisco il valore mentre se il dato 
# non è presente non inserisco la proprietà.
# Per la chiave del nodo (in questo caso il nome del pokemon) non faccio il controllo in quanto su neo4j ho inserito il vincolo di KEY che implica che la proprietà esista 
# e che sia unica 

# Costruisco la query con diversi passaggi in quanto per la parte centrale uso un ciclo per crearmi i controlli sulle varie colonne del dataset
pok_import_csv = "LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/rmarconzini/Dataset/main/df_pokemon_import.csv' AS row"
pok_key = "MERGE (p:Pokemon {Name: row.Name})"

pok_nokey = pok.drop('Name', axis=1)
pok_nokey=pok_nokey.columns
pok_nokey = pok_nokey.values.tolist()
pok_execution_commands = []
for i in pok_nokey:
    neo4j_create_statemenet ="FOREACH(ignoreMe IN CASE WHEN trim(row."+ str(i)  + ")" + " <> \"\" THEN [1] ELSE [] END | SET p."+str(i)+" = row."+str(i) +")"
    pok_execution_commands.append(neo4j_create_statemenet)
pok_creation_node = '\n'.join(pok_execution_commands)
pok_final_query = str(pok_import_csv)+'\n'+str(pok_key)+'\n'+str(pok_creation_node)+'\n'+'RETURN p'
print(pok_final_query) # bisogna sistemare i nomi delle proprietà che devono essere conformi alle convenzioni neo4j

LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/rmarconzini/Dataset/main/df_pokemon_import.csv' AS row
MERGE (p:Pokemon {Name: row.Name})
FOREACH(ignoreMe IN CASE WHEN trim(row.NationalPokedexNumber) <> "" THEN [1] ELSE [] END | SET p.NationalPokedexNumber = row.NationalPokedexNumber)
FOREACH(ignoreMe IN CASE WHEN trim(row.Species) <> "" THEN [1] ELSE [] END | SET p.Species = row.Species)
FOREACH(ignoreMe IN CASE WHEN trim(row.Variant) <> "" THEN [1] ELSE [] END | SET p.Variant = row.Variant)
FOREACH(ignoreMe IN CASE WHEN trim(row.Generation) <> "" THEN [1] ELSE [] END | SET p.Generation = row.Generation)
FOREACH(ignoreMe IN CASE WHEN trim(row.Rarity) <> "" THEN [1] ELSE [] END | SET p.Rarity = row.Rarity)
FOREACH(ignoreMe IN CASE WHEN trim(row.Type1) <> "" THEN [1] ELSE [] END | SET p.Type1 = row.Type1)
FOREACH(ignoreMe IN CASE WHEN trim(row.Type2) <> "" THEN [1] ELSE [] END | SET p.Type2 = row.Type2)
FOREACH(ignoreMe IN CASE WHEN trim(row.Total) <> "" THEN [1] ELSE [] E

In [39]:
# Aggiustamento della query in quanto l'import da python prevede questo tipo di formato (3 appici singoli) + inserisco i tipi integer dove necessario
# che mi viene più comodo farlo a mano rispetto che nel ciclo for

pok_import_nodes = '''
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/rmarconzini/Dataset/main/df_pokemon_import.csv' AS row
MERGE (p:Pokemon {Name: row.Name})
FOREACH(ignoreMe IN CASE WHEN trim(row.NationalPokedexNumber) <> "" THEN [1] ELSE [] END | SET p.NationalPokedexNumber = toInteger(row.NationalPokedexNumber))
FOREACH(ignoreMe IN CASE WHEN trim(row.Species) <> "" THEN [1] ELSE [] END | SET p.Species = row.Species)
FOREACH(ignoreMe IN CASE WHEN trim(row.Variant) <> "" THEN [1] ELSE [] END | SET p.Variant = row.Variant)
FOREACH(ignoreMe IN CASE WHEN trim(row.Generation) <> "" THEN [1] ELSE [] END | SET p.Generation = toInteger(row.Generation))
FOREACH(ignoreMe IN CASE WHEN trim(row.Rarity) <> "" THEN [1] ELSE [] END | SET p.Rarity = row.Rarity)
FOREACH(ignoreMe IN CASE WHEN trim(row.Type1) <> "" THEN [1] ELSE [] END | SET p.Type1 = row.Type1)
FOREACH(ignoreMe IN CASE WHEN trim(row.Type2) <> "" THEN [1] ELSE [] END | SET p.Type2 = row.Type2)
FOREACH(ignoreMe IN CASE WHEN trim(row.Total) <> "" THEN [1] ELSE [] END | SET p.Total = toInteger(row.Total))
FOREACH(ignoreMe IN CASE WHEN trim(row.HP) <> "" THEN [1] ELSE [] END | SET p.HP = toInteger(row.HP))
FOREACH(ignoreMe IN CASE WHEN trim(row.Attack) <> "" THEN [1] ELSE [] END | SET p.Attack = toInteger(row.Attack))
FOREACH(ignoreMe IN CASE WHEN trim(row.Defense) <> "" THEN [1] ELSE [] END | SET p.Defense = toInteger(row.Defense))
FOREACH(ignoreMe IN CASE WHEN trim(row.SpAtk) <> "" THEN [1] ELSE [] END | SET p.SpAtk = toInteger(row.SpAtk))
FOREACH(ignoreMe IN CASE WHEN trim(row.SpDef) <> "" THEN [1] ELSE [] END | SET p.SpDef = toInteger(row.SpDef))
FOREACH(ignoreMe IN CASE WHEN trim(row.Speed) <> "" THEN [1] ELSE [] END | SET p.Speed = toInteger(row.Speed))
FOREACH(ignoreMe IN CASE WHEN trim(row.image_url) <> "" THEN [1] ELSE [] END | SET p.image_url = row.image_url)
FOREACH(ignoreMe IN CASE WHEN trim(row.Vgc2022Rules) <> "" THEN [1] ELSE [] END | SET p.Vgc2022Rules = row.Vgc2022Rules)
FOREACH(ignoreMe IN CASE WHEN trim(row.MonthlyUsageMarch2022) <> "" THEN [1] ELSE [] END | SET p.MonthlyUsageMarch2022 = toInteger(row.MonthlyUsageMarch2022))
FOREACH(ignoreMe IN CASE WHEN trim(row.UsagePercentMarch2022) <> "" THEN [1] ELSE [] END | SET p.UsagePercentMarch2022 = toInteger(row.UsagePercentMarch2022))
FOREACH(ignoreMe IN CASE WHEN trim(row.MonthlyRankMarch2022) <> "" THEN [1] ELSE [] END | SET p.MonthlyRankMarch2022 = toInteger(row.MonthlyRankMarch2022))
RETURN p
'''

### `MOVES nodes`

Vedere i commenti sul nodo Pokemon per le indicazioni sulla costruzione della query

#### Load and Processing

In [40]:
# Load csv
moves=pd.read_csv('C:/Users/marco/OneDrive/UNIMIB_DataScience/99-PROJECT/DataManagement2022_Project/Dataset integrati 2.0/df_moves.csv', sep=',')
moves.head()

,Name,Introducted_in,Type,Power,Acc.,PP,Damage_class,Effect,Prob. (%)
0,Absorb,1,Grass,20.0,100,25.0,Special,User recovers half the HP inflicted on opponent.,NaN
1,Accelerock,7,Rock,40.0,100,20.0,Physical,User attacks first.,NaN
2,Acid,1,Poison,40.0,100,30.0,Special,May lower opponent's Special Defense.,10.0
3,Acid Armor,1,Poison,NaN,NaN,20.0,Status,Sharply raises user's Defense.,NaN
4,Acid Spray,5,Poison,40.0,100,20.0,Special,Sharply lowers opponent's Special Defense.,100.0


In [41]:
moves=moves.rename(columns={'Introducted_in':'IntroductedIn', 'Acc.':'Acc', 'Damage_class':'DamageClass', 'Prob. (%)':'ProbPercentage'})
moves=moves.replace(np.nan, '',regex=True) # Tolgo i valori NaN

In [42]:
moves.head()

,Name,IntroductedIn,Type,Power,Acc,PP,DamageClass,Effect,ProbPercentage
0,Absorb,1,Grass,20.0,100,25.0,Special,User recovers half the HP inflicted on opponent.,
1,Accelerock,7,Rock,40.0,100,20.0,Physical,User attacks first.,
2,Acid,1,Poison,40.0,100,30.0,Special,May lower opponent's Special Defense.,10.0
3,Acid Armor,1,Poison,,,20.0,Status,Sharply raises user's Defense.,
4,Acid Spray,5,Poison,40.0,100,20.0,Special,Sharply lowers opponent's Special Defense.,100.0


In [ ]:
moves.to_csv('df_moves_import.csv')

#### Creazione delle query

##### Vincoli e indici

In [43]:
moves_constraint = '''
CREATE CONSTRAINT MovesNameKey IF NOT EXISTS
FOR (m:Moves)
REQUIRE m.Moves IS UNIQUE
'''

##### Nodi

In [44]:
moves_import_csv = "LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/rmarconzini/Dataset/main/df_moves_import.csv' AS row"
moves_key = "MERGE (m:Moves {Name: row.Name})"

moves_nokey = moves.drop('Name', axis=1)
moves_nokey=moves_nokey.columns
moves_nokey = moves_nokey.values.tolist()
moves_execution_commands = []
for i in moves_nokey:
    neo4j_create_statemenet ="FOREACH(ignoreMe IN CASE WHEN trim(row."+ str(i)  + ")" + " <> \"\" THEN [1] ELSE [] END | SET m."+str(i)+" = row."+str(i) +")"
    moves_execution_commands.append(neo4j_create_statemenet)
moves_creation_node = '\n'.join(moves_execution_commands)
moves_final_query = str(moves_import_csv)+'\n'+str(moves_key)+'\n'+str(moves_creation_node)+'\n'+'RETURN m'

In [45]:
print(moves_final_query)

LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/rmarconzini/Dataset/main/df_moves_import.csv' AS row
MERGE (m:Moves {Name: row.Name})
FOREACH(ignoreMe IN CASE WHEN trim(row.IntroductedIn) <> "" THEN [1] ELSE [] END | SET m.IntroductedIn = row.IntroductedIn)
FOREACH(ignoreMe IN CASE WHEN trim(row.Type) <> "" THEN [1] ELSE [] END | SET m.Type = row.Type)
FOREACH(ignoreMe IN CASE WHEN trim(row.Power) <> "" THEN [1] ELSE [] END | SET m.Power = row.Power)
FOREACH(ignoreMe IN CASE WHEN trim(row.Acc) <> "" THEN [1] ELSE [] END | SET m.Acc = row.Acc)
FOREACH(ignoreMe IN CASE WHEN trim(row.PP) <> "" THEN [1] ELSE [] END | SET m.PP = row.PP)
FOREACH(ignoreMe IN CASE WHEN trim(row.DamageClass) <> "" THEN [1] ELSE [] END | SET m.DamageClass = row.DamageClass)
FOREACH(ignoreMe IN CASE WHEN trim(row.Effect) <> "" THEN [1] ELSE [] END | SET m.Effect = row.Effect)
FOREACH(ignoreMe IN CASE WHEN trim(row.ProbPercentage) <> "" THEN [1] ELSE [] END | SET m.ProbPercentage = row.ProbPercentage

In [46]:
# Sistemo la query
moves_import_nodes = '''
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/rmarconzini/Dataset/main/df_moves_import.csv' AS row
MERGE (m:Moves {Name: row.Name})
FOREACH(ignoreMe IN CASE WHEN trim(row.IntroductedIn) <> "" THEN [1] ELSE [] END | SET m.IntroductedIn = toInteger(row.IntroductedIn))
FOREACH(ignoreMe IN CASE WHEN trim(row.Type) <> "" THEN [1] ELSE [] END | SET m.Type = row.Type)
FOREACH(ignoreMe IN CASE WHEN trim(row.Power) <> "" THEN [1] ELSE [] END | SET m.Power = toInteger(row.Power))
FOREACH(ignoreMe IN CASE WHEN trim(row.Acc) <> "" THEN [1] ELSE [] END | SET m.Acc = toInteger(row.Acc))
FOREACH(ignoreMe IN CASE WHEN trim(row.PP) <> "" THEN [1] ELSE [] END | SET m.PP = toInteger(row.PP))
FOREACH(ignoreMe IN CASE WHEN trim(row.DamageClass) <> "" THEN [1] ELSE [] END | SET m.DamageClass = row.DamageClass)
FOREACH(ignoreMe IN CASE WHEN trim(row.Effect) <> "" THEN [1] ELSE [] END | SET m.Effect = row.Effect)
FOREACH(ignoreMe IN CASE WHEN trim(row.ProbPercentage) <> "" THEN [1] ELSE [] END | SET m.ProbPercentage = toInteger(row.ProbPercentage))
RETURN m
'''

### `TYPES nodes`

Vedere i commenti sul nodo Pokemon per le indicazioni sulla costruzione della query

#### Load and Processing

In [47]:
# Load csv
types=pd.read_csv('C:/Users/marco/OneDrive/UNIMIB_DataScience/99-PROJECT/DataManagement2022_Project/Dataset integrati 2.0/df_types.csv', sep=',')
types.head()

,Name,Generation
0,normal,1
1,fighting,1
2,flying,1
3,poison,1
4,ground,1


In [48]:
types=types.replace(np.nan, '',regex=True)
types.head()

In [ ]:
types.to_csv('df_types_import.csv')

#### Creazione delle query

##### Vincoli e indici

In [50]:
types_constraint = '''
CREATE CONSTRAINT TypesNameKey IF NOT EXISTS
FOR (t:Type)
REQUIRE t.type IS UNIQUE
'''

##### Nodi

In [51]:
types_import_csv = "LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/rmarconzini/Dataset/main/df_types_import.csv' AS row"
types_key = "MERGE (t:Type {Name: row.Name})"

types_nokey = types.drop('Name', axis=1)
types_nokey=types_nokey.columns
types_nokey = types_nokey.values.tolist()
types_execution_commands = []
for i in types_nokey:
    neo4j_create_statemenet ="FOREACH(ignoreMe IN CASE WHEN trim(row."+ str(i)  + ")" + " <> \"\" THEN [1] ELSE [] END | SET t."+str(i)+" = row."+str(i) +")"
    types_execution_commands.append(neo4j_create_statemenet)
types_creation_node = '\n'.join(types_execution_commands)
types_final_query = str(types_import_csv)+'\n'+str(types_key)+'\n'+str(types_creation_node)+'\n'+'RETURN t'

In [52]:
print(types_final_query)

LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/rmarconzini/Dataset/main/df_types_import.csv' AS row
MERGE (t:Type {Name: row.Name})
FOREACH(ignoreMe IN CASE WHEN trim(row.Generation) <> "" THEN [1] ELSE [] END | SET t.Generation = row.Generation)
RETURN t


In [53]:
types_import_nodes = '''
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/rmarconzini/Dataset/main/df_types_import.csv' AS row
MERGE (t:Type {Name: row.Name})
FOREACH(ignoreMe IN CASE WHEN trim(row.Generation) <> "" THEN [1] ELSE [] END | SET t.Generation = row.Generation)
RETURN t
'''

### `ITEM nodes`

Vedere i commenti sul nodo Pokemon per le indicazioni sulla costruzione della query

#### Load and Processing

In [3]:
# Load csv
items=pd.read_csv('C:/Users/marco/OneDrive/UNIMIB_DataScience/99-PROJECT/DataManagement2022_Project/Dataset integrati 2.0/df_items.csv', sep=',')
items.head()

,Name,Effect (alt.),Effect,Usage Attributes,Specific Category,General Category,image_url
0,Ability Capsule,Switches a Pokémon between its two possible (n...,A capsule that allows a Pokémon with two Abili...,NaN,Vitamins,Hold items,https://img.pokemondb.net/sprites/items/abilit...
1,Ability Patch,NaN,Changes a Pokémon's ability to its Hidden Abil...,NaN,Vitamins,General items,https://img.pokemondb.net/s.png
2,Ability Urge,Forcibly activates a friendly Pokémon's ability.,"When used, it activates the Ability of an ally...",NaN,Miracle Shooter,Battle items,https://img.pokemondb.net/sprites/items/abilit...
3,Abomasite,Held: Allows Abomasnow to Mega Evolve into Meg...,Enables Abomasnow to Mega Evolve during battle.,NaN,Mega Stones,Hold items,https://img.pokemondb.net/sprites/items/abomas...
4,Absolite,Held: Allows Absol to Mega Evolve into Mega Ab...,Enables Absol to Mega Evolve during battle.,NaN,Mega Stones,Hold items,https://img.pokemondb.net/sprites/items/absoli...


In [15]:
# Mi adeguo alle convenzioni di Neo4j
items=items.rename(columns={'Effect (alt.)':'AlternativeEffect', 'Usage Attributes':'UsageAttributes', 'Specific Category':'SpecificCategory', 'General Category':'GeneralCategory'})
items=items.replace(np.nan, '',regex=True)  # Tolgo valori Nul
# Per la colonna 'UsageAttributes' tolgo tutti i caratteri speciali e al posto della virgole metto il punto e virgola per separare i vari valori
# userò poi la funzione 'split' di chyper all'interno della query
items['UsageAttributes']=items['UsageAttributes'].replace(["\[","\]","'"], "",regex=True)
items['UsageAttributes']=items['UsageAttributes'].replace(',',";", regex=True)
display(items)


NameError: name 'items' is not defined

In [24]:
items.to_csv('df_items_import.csv')

#### Creazione delle query

##### Vincoli e indici

In [27]:
items_constraint='''
CREATE CONSTRAINT ItemIdKey IF NOT EXISTS
FOR (i:Item)
REQUIRE i.Id IS UNIQUE
'''

##### Nodi

In [85]:
items_import_csv = "LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/rmarconzini/Dataset/main/df_items_import.csv' AS row"
items_key = "MERGE (i:Item {Name: row.Name})"

items_nokey = items.drop('Name', axis=1)
items_nokey=items_nokey.columns
items_nokey =items_nokey.values.tolist()
items_execution_commands = []
for i in items_nokey:
    neo4j_create_statemenet ="FOREACH(ignoreMe IN CASE WHEN trim(row."+ str(i)  + ")" + " <> \"\" THEN [1] ELSE [] END | SET i."+str(i)+" = row."+str(i) +")"
    items_execution_commands.append(neo4j_create_statemenet)
items_creation_node = '\n'.join(items_execution_commands)
items_final_query = str(items_import_csv)+'\n'+str(items_key)+'\n'+str(items_creation_node)+'\n'+'RETURN i'

In [86]:
print(items_final_query)

LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/rmarconzini/Dataset/main/df_items_import.csv' AS row
MERGE (i:Item {Name: row.Name})
FOREACH(ignoreMe IN CASE WHEN trim(row.AlternativeEffect) <> "" THEN [1] ELSE [] END | SET i.AlternativeEffect = row.AlternativeEffect)
FOREACH(ignoreMe IN CASE WHEN trim(row.Effect) <> "" THEN [1] ELSE [] END | SET i.Effect = row.Effect)
FOREACH(ignoreMe IN CASE WHEN trim(row.UsageAttributes) <> "" THEN [1] ELSE [] END | SET i.UsageAttributes = row.UsageAttributes)
FOREACH(ignoreMe IN CASE WHEN trim(row.SpecificCategory) <> "" THEN [1] ELSE [] END | SET i.SpecificCategory = row.SpecificCategory)
FOREACH(ignoreMe IN CASE WHEN trim(row.GeneralCategory) <> "" THEN [1] ELSE [] END | SET i.GeneralCategory = row.GeneralCategory)
FOREACH(ignoreMe IN CASE WHEN trim(row.image_url) <> "" THEN [1] ELSE [] END | SET i.image_url = row.image_url)
RETURN i


In [26]:
# Sistemazione della query e uso della funzione 'split' per la proprietà UsageAttributes
items_import_nodes = '''
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/rmarconzini/Dataset/main/df_items_import.csv' AS row
MERGE (i:Item {Name: row.Name})
FOREACH(ignoreMe IN CASE WHEN trim(row.AlternativeEffect) <> "" THEN [1] ELSE [] END | SET i.AlternativeEffect = row.AlternativeEffect)
FOREACH(ignoreMe IN CASE WHEN trim(row.Effect) <> "" THEN [1] ELSE [] END | SET i.Effect = row.Effect)
FOREACH(ignoreMe IN CASE WHEN trim(row.UsageAttributes) <> "" THEN [1] ELSE [] END | SET i.UsageAttributes = split(row.UsageAttributes, ";"))
FOREACH(ignoreMe IN CASE WHEN trim(row.SpecificCategory) <> "" THEN [1] ELSE [] END | SET i.SpecificCategory = row.SpecificCategory)
FOREACH(ignoreMe IN CASE WHEN trim(row.GeneralCategory) <> "" THEN [1] ELSE [] END | SET i.GeneralCategory = row.GeneralCategory)
FOREACH(ignoreMe IN CASE WHEN trim(row.image_url) <> "" THEN [1] ELSE [] END | SET i.image_url = row.image_url)
RETURN i
'''

### `ABILITIES nodes`

Vedere i commenti sul nodo Pokemon per le indicazioni sulla costruzione della query

#### Load and processing

In [34]:
# Load csv
abilities=pd.read_csv('C:/Users/marco/OneDrive/UNIMIB_DataScience/99-PROJECT/DataManagement2022_Project/Dataset integrati 2.0/df_abilities.csv', sep=',')
abilities.head()

,Name,Generation,Description
0,Adaptability,4,Powers up moves of the same type.
1,Aerilate,6,Turns Normal-type moves into Flying-type moves.
2,Aftermath,4,Damages the attacker landing the finishing hit.
3,Air Lock,3,Eliminates the effects of weather.
4,Analytic,5,Boosts move power when the Pokémon moves last.


In [ ]:
abilities.to_csv('df_abilities_import.csv')

#### Creazione delle query

##### Vincoli e indici

In [41]:
abilities_constraint= '''
CREATE CONSTRAINT AbilitiesNameKey IF NOT EXISTS
FOR (a:Abilities)
REQUIRE a.Name IS UNIQUE
'''

##### Nodi

In [45]:
abilities_import_csv = "LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/rmarconzini/Dataset/main/df_abilities_import.csv' AS row"
abilities_key = "MERGE (a:Abilities {Name: row.Name})"

abilities_nokey = abilities.drop('Name', axis=1)
abilities_nokey=abilities_nokey.columns
abilities_nokey =abilities_nokey.values.tolist()
abilities_execution_commands = []

for i in abilities_nokey:
    neo4j_create_statemenet ="FOREACH(ignoreMe IN CASE WHEN trim(row."+ str(i)  + ")" + " <> \"\" THEN [1] ELSE [] END | SET a."+str(i)+" = row."+str(i) +")"
    abilities_execution_commands.append(neo4j_create_statemenet)
abilities_creation_node = '\n'.join(abilities_execution_commands)
abilities_final_query = str(abilities_import_csv)+'\n'+str(abilities_key)+'\n'+str(abilities_creation_node)+'\n'+'RETURN a'

In [46]:
print(abilities_final_query)

LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/rmarconzini/Dataset/main/df_abilities_import.csv' AS row
MERGE (a:Abilities {Name: row.Name})
FOREACH(ignoreMe IN CASE WHEN trim(row.Generation) <> "" THEN [1] ELSE [] END | SET a.Generation = row.Generation)
FOREACH(ignoreMe IN CASE WHEN trim(row.Description) <> "" THEN [1] ELSE [] END | SET a.Description = row.Description)
RETURN a


In [47]:
abilities_import_nodes ='''
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/rmarconzini/Dataset/main/df_abilities_import.csv' AS row
MERGE (a:Abilities {Name: row.Name})
FOREACH(ignoreMe IN CASE WHEN trim(row.Generation) <> "" THEN [1] ELSE [] END | SET a.Generation = row.Generation)
FOREACH(ignoreMe IN CASE WHEN trim(row.Description) <> "" THEN [1] ELSE [] END | SET a.Description = row.Description)
RETURN a
'''

### `NATURES nodes`

Vedere i commenti sul nodo Pokemon per le indicazioni sulla costruzione della query

#### Load and Processing

In [49]:
# Laod csv
natures=pd.read_csv('C:/Users/marco/OneDrive/UNIMIB_DataScience/99-PROJECT/DataManagement2022_Project/Dataset integrati 2.0/df_natures.csv', sep=',')
natures.head()

,Nature,Increases,Decreases,Likes_berrie,Dislikes_berrie
0,Adamant,Attack,Sp. Atk,Spicy,Dry
1,Bashful,Sp. Atk,Sp. Atk,Dry,Dry
2,Bold,Defense,Attack,Sour,Spicy
3,Brave,Attack,Speed,Spicy,Sweet
4,Calm,Sp. Def,Attack,Bitter,Spicy


In [67]:
# Mi adeguo alle convenzioni di Neo4j
natures=natures.rename(columns={'Nature':'NatureName', 'Likes_berrie':'LikesBerrie', 'Dislikes_berrie':'DislikesBerrie'})
display(natures)

,NatureName,Increases,Decreases,LikesBerrie,DislikesBerrie
0,Adamant,Attack,Sp. Atk,Spicy,Dry
1,Bashful,Sp. Atk,Sp. Atk,Dry,Dry
2,Bold,Defense,Attack,Sour,Spicy
3,Brave,Attack,Speed,Spicy,Sweet
4,Calm,Sp. Def,Attack,Bitter,Spicy
5,Careful,Sp. Def,Sp. Atk,Bitter,Dry
6,Docile,Defense,Defense,Sour,Sour
7,Gentle,Sp. Def,Defense,Bitter,Sour
8,Hardy,Attack,Attack,Spicy,Spicy
9,Hasty,Speed,Defense,Sweet,Sour


In [68]:
natures.to_csv('df_natures_import.csv')

#### Creazione delle query

##### Vincoli e indici

In [63]:
natures_constraint ='''
CREATE CONSTRAINT NaturesNameKey IF NOT EXISTS
FOR (n:Nature)
REQUIRE n.NatureName IS UNIQUE
'''

##### Nodi

In [59]:
natures_import_csv = "LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/rmarconzini/Dataset/main/df_natures_import.csv' AS row"
natures_key = "MERGE (n:Nature {Name: row.NatureName})"

natures_nokey = natures.drop('NatureName', axis=1)
natures_nokey=natures_nokey.columns
natures_nokey =natures_nokey.values.tolist()
natures_execution_commands = []
for i in natures_nokey:
    neo4j_create_statemenet ="FOREACH(ignoreMe IN CASE WHEN trim(row."+ str(i)  + ")" + " <> \"\" THEN [1] ELSE [] END | SET n."+str(i)+" = row."+str(i) +")"
    natures_execution_commands.append(neo4j_create_statemenet)
natures_creation_node = '\n'.join(natures_execution_commands)
natures_final_query = str(natures_import_csv)+'\n'+str(natures_key)+'\n'+str(natures_creation_node)+'\n'+'RETURN n'

In [60]:
print(natures_final_query)

LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/rmarconzini/Dataset/main/df_natures_import.csv' AS row
MERGE (n:Nature {Name: row.NatureName})
FOREACH(ignoreMe IN CASE WHEN trim(row.Increases) <> "" THEN [1] ELSE [] END | SET n.Increases = row.Increases)
FOREACH(ignoreMe IN CASE WHEN trim(row.Decreases) <> "" THEN [1] ELSE [] END | SET n.Decreases = row.Decreases)
FOREACH(ignoreMe IN CASE WHEN trim(row.LikesBerrie) <> "" THEN [1] ELSE [] END | SET n.LikesBerrie = row.LikesBerrie)
FOREACH(ignoreMe IN CASE WHEN trim(row.DislikesBerrie) <> "" THEN [1] ELSE [] END | SET n.DislikesBerrie = row.DislikesBerrie)
RETURN n


In [61]:
natures_import_nodes='''
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/rmarconzini/Dataset/main/df_natures_import.csv' AS row
MERGE (n:Nature {Name: row.NatureName})
FOREACH(ignoreMe IN CASE WHEN trim(row.Increases) <> "" THEN [1] ELSE [] END | SET n.Increases = row.Increases)
FOREACH(ignoreMe IN CASE WHEN trim(row.Decreases) <> "" THEN [1] ELSE [] END | SET n.Decreases = row.Decreases)
FOREACH(ignoreMe IN CASE WHEN trim(row.LikesBerrie) <> "" THEN [1] ELSE [] END | SET n.LikesBerrie = row.LikesBerrie)
FOREACH(ignoreMe IN CASE WHEN trim(row.DislikesBerrie) <> "" THEN [1] ELSE [] END | SET n.DislikesBerrie = row.DislikesBerrie)
RETURN n
'''

## Relationships

### `Pokemon -[IS_OF_TYPE]-> Type`

In [79]:
# Load csv
is_of_type = pd.read_csv('C:/Users/marco/OneDrive/UNIMIB_DataScience/99-PROJECT/DataManagement2022_Project/Dataset integrati 2.0/ponte_pokemon_type_IS_OF_TYPE.csv', sep=',')
is_of_type.head()

,Name,Type
0,Bulbasaur,Grass
1,Bulbasaur,Poison
2,Ivysaur,Grass
3,Ivysaur,Poison
4,Venusaur,Grass


In [84]:
# Essendo che il dataset dei nodi "type" contiene la prima lettera minuscola, mentre il dataset ponte maiuscola, trasformo la colonna "Type" in tutte lettere minuscole
is_of_type['Type']=is_of_type['Type'].str.lower()
display(is_of_type)

,Name,Type
0,Bulbasaur,grass
1,Bulbasaur,poison
2,Ivysaur,grass
3,Ivysaur,poison
4,Venusaur,grass
...,...,...
1681,Calyrex,grass
1682,Calyrex Shadow Rider,psychic
1683,Calyrex Shadow Rider,ghost
1684,Calyrex Ice Rider,psychic


In [85]:
is_of_type.to_csv('ponte_pokemon_type_IS_OF_TYPE_import.csv')

In [86]:
is_of_type_import_rel ='''
LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/rmarconzini/Dataset/main/ponte_pokemon_type_IS_OF_TYPE_import.csv" AS row
MATCH (p:Pokemon {Name: row.Name}), (t:Type {Name: row.Type})
CREATE (p)-[:IS_OF_TYPE]->(t)
'''

### `Pokemon -[EVOLVES_INTO]-> Pokemon`


Il dataset creato da Giorgio prevede la relazione al contrario. Per evitare di creare delle relazioni cicliche tra nodi importerò la relazione "EVOLVES_INTO" e non "EVOLVES_FROM"

In [95]:
# Load csv
evolves_into = pd.read_csv('C:/Users/marco/OneDrive/UNIMIB_DataScience/99-PROJECT/DataManagement2022_Project/Dataset integrati 2.0/ponte_pokemon_pokemon_EVOLVES_FROM.csv', sep=',')
evolves_into = evolves_into.rename(columns={'Name':'Evolves_into', 'Evolves_from':'Name'})
display(evolves_into)

,Evolves_into,Name
0,Ivysaur,Bulbasaur
1,Venusaur,Ivysaur
2,Mega Venusaur,Venusaur
3,Venusaur Gigantamax,Venusaur
4,Charmeleon,Charmander
...,...,...
550,Dragapult,Drakloak
551,Urshifu (Single Strike) Gigantamax,Urshifu (Single Strike)
552,Urshifu (Rapid Strike) Gigantamax,Urshifu (Rapid Strike)
553,Urshifu Single Strike Style,Kubfu


In [96]:
evolves_into.to_csv('ponte_pokemon_pokemon_EVOLVES_INTO_import.csv')

In [91]:
evolves_into_rel ='''
LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/rmarconzini/Dataset/main/ponte_pokemon_pokemon_EVOLVES_INTO_import.csv" AS row
MATCH (p:Pokemon {Name: row.Name}), (e:Pokemon {Name: row.Evolves_into})
CREATE (p)-[:EVOLVES_INTO]->(e)
'''

### `Pokemon -[HAS_VARIANT]-> Pokemon`


In [92]:
has_variant = pd.read_csv('C:/Users/marco/OneDrive/UNIMIB_DataScience/99-PROJECT/DataManagement2022_Project/Dataset integrati 2.0/ponte_pokemon_pokemon_HAS_VARIANT.csv', sep=',')
display(has_variant)

,Non-Variant Pkm Name,Variant Pkm Name
0,Venusaur,Mega Venusaur
1,Venusaur,Venusaur Gigantamax
2,Charizard,Mega Charizard Y
3,Charizard,Charizard Gigantamax
4,Charizard,Mega Charizard X
...,...,...
148,Copperajah,Copperajah Gigantamax
149,Duraludon,Duraludon Gigantamax
150,Eternatus,Eternatus Eternamax
151,Calyrex,Calyrex Shadow Rider


In [93]:
has_variant=has_variant.rename(columns={'Non-Variant Pkm Name':'NoVariant','Variant Pkm Name':'Variant' })
display(has_variant)

,NoVariant,Variant
0,Venusaur,Mega Venusaur
1,Venusaur,Venusaur Gigantamax
2,Charizard,Mega Charizard Y
3,Charizard,Charizard Gigantamax
4,Charizard,Mega Charizard X
...,...,...
148,Copperajah,Copperajah Gigantamax
149,Duraludon,Duraludon Gigantamax
150,Eternatus,Eternatus Eternamax
151,Calyrex,Calyrex Shadow Rider


In [94]:
has_variant.to_csv('ponte_pokemon_pokemon_HAS_VARIANT_import.csv')

In [ ]:
has_variant_rel ='''
LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/rmarconzini/Dataset/main/ponte_pokemon_pokemon_HAS_VARIANT_import.csv" AS row
MATCH (p:Pokemon {Name: row.NoVariant}), (v:Pokemon {Name: row.Variant})
CREATE (p)-[:HAS_VARIANT]->(v)
'''

### `Pokemon -[USED_IN_TEAM_WITH]-> Pokemon`

In [13]:
used_in_team_with = pd.read_csv('C:/Users/marco/OneDrive/UNIMIB_DataScience/99-PROJECT/DataManagement2022_Project/Dataset integrati 2.0/ponte_pokemon_pokemon_USED_IN_TEAM_WITH.csv', sep=',')
display(used_in_team_with)

,Use_Percentage (%),Pokemon,Teammate
0,54.189%,Zacian Crowned Sword,Incineroar
1,37.766%,Kyogre,Incineroar
2,56.035%,Grimmsnarl,Incineroar
3,47.343%,Regieleki,Incineroar
4,61.378%,Rillaboom,Incineroar
...,...,...,...
3406,50.083%,Noctowl,Claydol
3407,6.603%,Corsola,Flygon
3408,0.335%,Mime Jr.,Scrafty
3409,0.014%,Gourgeist Large Size,Scrafty


In [14]:
used_in_team_with =used_in_team_with.rename(columns={'Use_Percentage (%)':"UsePercentage"}) 
used_in_team_with['UsePercentage']=used_in_team_with['UsePercentage'].replace('%', '', regex=True) 
display(used_in_team_with)

,UsePercentage,Pokemon,Teammate
0,54.189,Zacian Crowned Sword,Incineroar
1,37.766,Kyogre,Incineroar
2,56.035,Grimmsnarl,Incineroar
3,47.343,Regieleki,Incineroar
4,61.378,Rillaboom,Incineroar
...,...,...,...
3406,50.083,Noctowl,Claydol
3407,6.603,Corsola,Flygon
3408,0.335,Mime Jr.,Scrafty
3409,0.014,Gourgeist Large Size,Scrafty


In [101]:
used_in_team_with.to_csv('ponte_pokemon_pokemon_USED_IN_TEAM_WITH_import.csv')

In [15]:
used_in_team_with_rel ='''
LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/rmarconzini/Dataset/main/ponte_pokemon_pokemon_USED_IN_TEAM_WITH_import.csv" AS row
MATCH (p:Pokemon {Name: row.Pokemon}), (t:Pokemon {Name: row.Teammate})
CREATE (p)-[:USED_IN_TEAM_WITH {UsePercentage: toFloat(row.UsePercentage)}]->(t)
'''


### `Pokemon -[MAY_LEARN]-> MOVES`

In [102]:
may_learn = pd.read_csv('C:/Users/marco/OneDrive/UNIMIB_DataScience/99-PROJECT/DataManagement2022_Project/Dataset integrati 2.0/ponte_pokemon_moves_MAY_LEARN.csv', sep=',')
display(may_learn)

,Move,Pokemon
0,Absorb,Zubat
1,Acrobatics,Zubat
2,Aerial Ace,Zubat
3,Agility,Zubat
4,Air Cutter,Zubat
...,...,...
68100,Supersonic,Blipbug
68101,Sketch,Smeargle
68102,Splash,Cosmog
68103,Teleport,Cosmog


In [104]:
may_learn_rel ='''
LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/rmarconzini/Dataset/main/ponte_pokemon_moves_MAY_LEARN_import.csv" AS row
MATCH (p:Pokemon {Name: row.Pokemon}), (m:Moves {Name: row.Move})
CREATE (p)-[:MAY_LEARN]->(m)
'''

### `GigantaMaxPokemon -[MAY_LEARN_GMAX]-> MaxMove`

In [107]:
may_learn_gmax = pd.read_csv('C:/Users/marco/OneDrive/UNIMIB_DataScience/99-PROJECT/DataManagement2022_Project/Dataset integrati 2.0/ponte_moves_pokemon_GMAX_MOVE.csv', sep=',')
may_learn_gmax =may_learn_gmax.rename(columns={'Max Move':'MaxMove', 'Gigantamax Pokémon':'GigantamaxPokemon'})
display(may_learn_gmax)


,MaxMove,GigantamaxPokemon
0,G-Max Vine Lash,Venusaur Gigantamax
1,G-Max Wildfire,Charizard Gigantamax
2,G-Max Cannonade,Blastoise Gigantamax
3,G-Max Befuddle,Butterfree Gigantamax
4,G-Max Volt Crash,Pikachu Gigantamax
5,G-Max Gold Rush,Meowth Gigantamax
6,G-Max Chi Strike,Machamp Gigantamax
7,G-Max Terror,Gengar Gigantamax
8,G-Max Foam Burst,Kingler Gigantamax
9,G-Max Resonance,Lapras Gigantamax


In [110]:
may_learn_gmax.to_csv('ponte_moves_pokemon_GMAX_MOVE_import.csv')

In [108]:
may_learn_gmax_rel ='''
LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/rmarconzini/Dataset/main/ponte_moves_pokemon_GMAX_MOVE_import.csv" AS row
MATCH (p:Pokemon {Name: row.GigantamaxPokemon}), (m:Moves {Name: row.MaxMove})
CREATE (p)-[:MAY_LEARN_GMAX]->(m)
'''

### `Pokemon -[USED_WITH_ITEM]-> Item`

In [19]:
used_with_item = pd.read_csv('C:/Users/marco/OneDrive/UNIMIB_DataScience/99-PROJECT/DataManagement2022_Project/Dataset integrati 2.0/ponte_pokemon_item_USED_WITH_ITEM.csv', sep=',')
display(used_with_item)

,Item,Use_Percentage (%),Pokemon
0,Rusted Sword,100.000%,Zacian Crowned Sword
1,Shuca Berry,36.082%,Incineroar
2,Safety Goggles,23.390%,Incineroar
3,Sitrus Berry,14.031%,Incineroar
4,Focus Sash,8.994%,Incineroar
...,...,...,...
1887,Other,0.734%,Corsola
1888,Focus Sash,99.860%,Mime Jr.
1889,Other,0.140%,Mime Jr.
1890,Figy Berry,99.986%,Gourgeist Large Size


In [20]:
used_with_item =used_with_item.rename(columns={'Use_Percentage (%)':"UsePercentage"}) 
used_with_item['UsePercentage']=used_with_item['UsePercentage'].replace('%', '', regex=True) 
display(used_with_item)

,Item,UsePercentage,Pokemon
0,Rusted Sword,100.000,Zacian Crowned Sword
1,Shuca Berry,36.082,Incineroar
2,Safety Goggles,23.390,Incineroar
3,Sitrus Berry,14.031,Incineroar
4,Focus Sash,8.994,Incineroar
...,...,...,...
1887,Other,0.734,Corsola
1888,Focus Sash,99.860,Mime Jr.
1889,Other,0.140,Mime Jr.
1890,Figy Berry,99.986,Gourgeist Large Size


In [21]:
used_with_item.to_csv('ponte_pokemon_item_USED_WITH_ITEM_import.csv')

In [17]:
used_with_item_rel ='''
LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/rmarconzini/Dataset/main/ponte_pokemon_item_USED_WITH_ITEM_import.csv" AS row
MATCH (p:Pokemon {Name: row.Pokemon}), (i:Item {Name: row.Item})
CREATE (p)-[:USED_WITH_ITEM {UsePercentage: toFloat(row.UsePercentage)}]->(i)
'''

### `Pokemon -[USED_WITH_ABILITY]-> Ability`

In [22]:
used_with_ability = pd.read_csv('C:/Users/marco/OneDrive/UNIMIB_DataScience/99-PROJECT/DataManagement2022_Project/Dataset integrati 2.0/ponte_pokemon_ability_MAY_HAS.csv', sep=',')
display(used_with_ability)

,Ability,Hidden,Pokemon
0,Adaptability,False,Eevee
1,Anticipation,True,Eevee
2,Run Away,False,Eevee
3,Adaptability,True,Corphish
4,Hyper Cutter,False,Corphish
...,...,...,...
2170,Water Bubble,False,Dewpider
2171,Water Absorb,True,Araquanid
2172,Water Bubble,False,Araquanid
2173,Wimp Out,False,Wimpod


In [32]:
# Per i dati di tipo boolean Neo4j richiede che in input vengano dati valori 0/1, quindi convento
used_with_ability['Hidden']=used_with_ability['Hidden'].astype(int)
display(used_with_ability['Hidden'])

0       0
1       1
2       0
3       1
4       0
       ..
2170    0
2171    1
2172    0
2173    0
2174    0
Name: Hidden, Length: 2175, dtype: int32

In [33]:
used_with_ability.to_csv("ponte_pokemon_ability_MAY_HAS_import.csv")

In [69]:
used_with_ability_rel ='''
LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/rmarconzini/Dataset/main/ponte_pokemon_ability_MAY_HAS_import.csv" AS row
MATCH (p:Pokemon {Name: row.Pokemon}), (a:Abilities {Name: row.Ability})
CREATE (p)-[:USED_WITH_ABILITY {IsHidden: (CASE row.Hidden WHEN 1 THEN true ELSE false END)}]->(a)
'''

### `Pokemon -[COMMON_SPREAD]-> Natures`

In [63]:
common_spread = pd.read_csv('C:/Users/marco/OneDrive/UNIMIB_DataScience/99-PROJECT/DataManagement2022_Project/Dataset integrati 2.0/ponte_pokemon_nature_COMMON_SPREAD.csv', sep=',')
display(common_spread)

,Nature,HP/Atk/Def/SpA/SpD/Spe,Use_Percentage (%),Pokemon
0,Jolly,0/252/0/0/4/252,10.388%,Zacian Crowned Sword
1,Adamant,252/28/4/0/12/212,4.504%,Zacian Crowned Sword
2,Jolly,4/252/0/0/0/252,4.419%,Zacian Crowned Sword
3,Adamant,188/164/4/0/4/148,2.891%,Zacian Crowned Sword
4,Jolly,0/252/4/0/0/252,2.456%,Zacian Crowned Sword
...,...,...,...,...
6814,Sassy,252/4/0/0/252/0,0.000%,Gourgeist Large Size
6815,Relaxed,252/0/252/4/0/0,0.000%,Gourgeist Large Size
6816,Quiet,252/0/0/252/4/0,0.000%,Gourgeist Large Size
6817,Adamant,252/60/0/0/196/0,0.000%,Gourgeist Large Size


In [64]:
common_spread = common_spread.rename(columns={'Use_Percentage (%)':'UsePercentage'})
common_spread['UsePercentage']=common_spread['UsePercentage'].replace('%','', regex=True)

# Neo4j non accetta che una label abbia caratteri speciali tipo '/'. Divido quindi le varie statistiche come fatto per il nodo "Pokemon"
common_spread=common_spread.astype({'HP/Atk/Def/SpA/SpD/Spe': 'string'})
common_spread[['Hp', 'Atk', 'Def', 'SpA', 'SpD', 'Spe']]=common_spread['HP/Atk/Def/SpA/SpD/Spe'].str.split('/', expand=True)
common_spread=common_spread.drop('HP/Atk/Def/SpA/SpD/Spe', axis=1)
display(common_spread)

,Nature,UsePercentage,Pokemon,Hp,Atk,Def,SpA,SpD,Spe
0,Jolly,10.388,Zacian Crowned Sword,0,252,0,0,4,252
1,Adamant,4.504,Zacian Crowned Sword,252,28,4,0,12,212
2,Jolly,4.419,Zacian Crowned Sword,4,252,0,0,0,252
3,Adamant,2.891,Zacian Crowned Sword,188,164,4,0,4,148
4,Jolly,2.456,Zacian Crowned Sword,0,252,4,0,0,252
...,...,...,...,...,...,...,...,...,...
6814,Sassy,0.000,Gourgeist Large Size,252,4,0,0,252,0
6815,Relaxed,0.000,Gourgeist Large Size,252,0,252,4,0,0
6816,Quiet,0.000,Gourgeist Large Size,252,0,0,252,4,0
6817,Adamant,0.000,Gourgeist Large Size,252,60,0,0,196,0


In [54]:
common_spread.to_csv('ponte_pokemon_nature_COMMON_SPREAD_import.csv')

In [65]:
common_spread_rel ='''
LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/rmarconzini/Dataset/main/ponte_pokemon_nature_COMMON_SPREAD_import.csv" AS row
MATCH (p:Pokemon {Name: row.Pokemon}), (n:Nature {Name: row.Nature})
CREATE (p)-[:COMMON_SPREAD {Hp: toInteger(row.Hp), Atk: toInteger(row.Atk), Def: toInteger(row.Def), SpA: toInteger(row.SpA), SpD: toInteger(row.SpD), Spe: toInteger(row.Spe), UsePercentage: toInteger(row.UsePercentage)}] -> (n)
'''

### `Pokemon -[USED_WITH_MOVE]-> Moves`

In [55]:
used_with_move = pd.read_csv('C:/Users/marco/OneDrive/UNIMIB_DataScience/99-PROJECT/DataManagement2022_Project/Dataset integrati 2.0/ponte_pokemon_move_USED_WITH_MOVE.csv', sep=',')
display(used_with_move)

,Move,Use_Percentage (%),Pokemon,Name
0,Behemoth Blade,99.996%,Zacian Crowned Sword,Behemoth Blade
1,Protect,98.672%,Zacian Crowned Sword,Protect
2,Protect,67.612%,Kyogre,Protect
3,Protect,72.095%,Regieleki,Protect
4,Protect,47.379%,Rillaboom,Protect
...,...,...,...,...
3266,Wrap,23.041%,Lickilicky,Wrap
3267,Thrash,17.229%,Lickilicky,Thrash
3268,Headbutt,33.562%,Dunsparce,Headbutt
3269,Sleep Talk,9.898%,Dunsparce,Sleep Talk


In [67]:
used_with_move = used_with_move.rename(columns={'Use_Percentage (%)':'UsePercentage'})
used_with_move['UsePercentage']=used_with_move['UsePercentage'].replace('%', '', regex=True)
used_with_move=used_with_move.drop('Name',axis=1)
display(used_with_move)

,Move,UsePercentage,Pokemon
0,Behemoth Blade,99.996,Zacian Crowned Sword
1,Protect,98.672,Zacian Crowned Sword
2,Protect,67.612,Kyogre
3,Protect,72.095,Regieleki
4,Protect,47.379,Rillaboom
...,...,...,...
3266,Wrap,23.041,Lickilicky
3267,Thrash,17.229,Lickilicky
3268,Headbutt,33.562,Dunsparce
3269,Sleep Talk,9.898,Dunsparce


In [62]:
used_with_move.to_csv('ponte_pokemon_move_USED_WITH_MOVE_import.csv')

In [68]:
used_with_move_rel ='''
LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/rmarconzini/Dataset/main/ponte_pokemon_move_USED_WITH_MOVE_import.csv" AS row
MATCH (p:Pokemon {Name: row.Pokemon}), (m:Moves {Name: row.Move})
CREATE (p)-[:USED_WITH_MOVE {UsePercentage: toInteger(row.UsePercentage)}] -> (m)
'''

### `Type -[MOVE_EFFECTIVENESS_ON_POKEMON]-> Type`

In [3]:
move_effectiveness_on_pokemon = pd.read_csv('C:/Users/marco/OneDrive/UNIMIB_DataScience/99-PROJECT/DataManagement2022_Project/Dataset integrati 2.0/ponte_type_type_MOVE_EFFECTIVENESS_ON_POKEMON.csv', sep=',')
display(move_effectiveness_on_pokemon)

,Atk. Move Type,Def. Pokemon Type,Damage Multiplier,Effectiveness
0,Normal,Normal,NaN,Normal (100%)
1,Fire,Normal,NaN,Normal (100%)
2,Water,Normal,NaN,Normal (100%)
3,Electric,Normal,NaN,Normal (100%)
4,Grass,Normal,NaN,Normal (100%)
...,...,...,...,...
319,Ghost,Fairy,NaN,Normal (100%)
320,Dragon,Fairy,0,Normal (100%)
321,Dark,Fairy,½,Not very effective (50%)
322,Steel,Fairy,2,Super-effective (200%)


In [4]:
move_effectiveness_on_pokemon =move_effectiveness_on_pokemon.rename(columns={'Atk. Move Type':'MoveType', 'Def. Pokemon Type':'PokemonType', 'Damage Multiplier':'DamageMultiplier'})
move_effectiveness_on_pokemon['DamageMultiplier']=move_effectiveness_on_pokemon['DamageMultiplier'].replace('½', "0.5", regex=True) # importo come float, quindi prima sistemo i caratteri ambigui e poi converto in float
display(move_effectiveness_on_pokemon)

,MoveType,PokemonType,DamageMultiplier,Effectiveness
0,Normal,Normal,NaN,Normal (100%)
1,Fire,Normal,NaN,Normal (100%)
2,Water,Normal,NaN,Normal (100%)
3,Electric,Normal,NaN,Normal (100%)
4,Grass,Normal,NaN,Normal (100%)
...,...,...,...,...
319,Ghost,Fairy,NaN,Normal (100%)
320,Dragon,Fairy,0,Normal (100%)
321,Dark,Fairy,0.5,Not very effective (50%)
322,Steel,Fairy,2,Super-effective (200%)


In [5]:
# Converto in float
move_effectiveness_on_pokemon['DamageMultiplier']=move_effectiveness_on_pokemon['DamageMultiplier'].apply(float)
display(move_effectiveness_on_pokemon)

,MoveType,PokemonType,DamageMultiplier,Effectiveness
0,Normal,Normal,NaN,Normal (100%)
1,Fire,Normal,NaN,Normal (100%)
2,Water,Normal,NaN,Normal (100%)
3,Electric,Normal,NaN,Normal (100%)
4,Grass,Normal,NaN,Normal (100%)
...,...,...,...,...
319,Ghost,Fairy,NaN,Normal (100%)
320,Dragon,Fairy,0.0,Normal (100%)
321,Dark,Fairy,0.5,Not very effective (50%)
322,Steel,Fairy,2.0,Super-effective (200%)


In [6]:
# Tolgo NaN
move_effectiveness_on_pokemon['DamageMultiplier']=move_effectiveness_on_pokemon['DamageMultiplier'].replace(np.nan, "",regex=True)
display(move_effectiveness_on_pokemon)

,MoveType,PokemonType,DamageMultiplier,Effectiveness
0,Normal,Normal,,Normal (100%)
1,Fire,Normal,,Normal (100%)
2,Water,Normal,,Normal (100%)
3,Electric,Normal,,Normal (100%)
4,Grass,Normal,,Normal (100%)
...,...,...,...,...
319,Ghost,Fairy,,Normal (100%)
320,Dragon,Fairy,0.0,Normal (100%)
321,Dark,Fairy,0.5,Not very effective (50%)
322,Steel,Fairy,2.0,Super-effective (200%)


In [8]:
# Converto i caratteri Maiuscoli in minuscoli
move_effectiveness_on_pokemon['MoveType']=move_effectiveness_on_pokemon['MoveType'].str.lower()
move_effectiveness_on_pokemon['PokemonType']=move_effectiveness_on_pokemon['PokemonType'].str.lower()
display(move_effectiveness_on_pokemon)

,MoveType,PokemonType,DamageMultiplier,Effectiveness
0,normal,normal,,Normal (100%)
1,fire,normal,,Normal (100%)
2,water,normal,,Normal (100%)
3,electric,normal,,Normal (100%)
4,grass,normal,,Normal (100%)
...,...,...,...,...
319,ghost,fairy,,Normal (100%)
320,dragon,fairy,0.0,Normal (100%)
321,dark,fairy,0.5,Not very effective (50%)
322,steel,fairy,2.0,Super-effective (200%)


In [9]:
move_effectiveness_on_pokemon.to_csv('ponte_type_type_MOVE_EFFECTIVENESS_ON_POKEMON_import.csv')

In [10]:
move_effectiveness_on_pokemon_rel= '''
LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/rmarconzini/Dataset/main/ponte_type_type_MOVE_EFFECTIVENESS_ON_POKEMON_import.csv" AS row
MATCH (m:Type {Name: row.MoveType}), (p:Type {Name: row.PokemonType})
CREATE (m)-[:MOVE_EFFECTIVENESS_ON_POKEMON {DamageMultiplier: toFloat(row.DamageMultiplier), Effectiveness: row.Effectiveness}] -> (p)
'''

### `Moves -[MOVES_IS_TYPE]-> Type`

In [12]:
moves_is_type = pd.read_csv('C:/Users/marco/OneDrive/UNIMIB_DataScience/99-PROJECT/DataManagement2022_Project/Dataset integrati 2.0/ponte_move_type_MOVES_IS_TYPE.csv', sep=',')
display(moves_is_type)

,Name,Type
0,Absorb,Grass
1,Accelerock,Rock
2,Acid,Poison
3,Acid Armor,Poison
4,Acid Spray,Poison
...,...,...
816,G-Max Finale,Fairy
817,G-Max Steelsurge,Steel
818,G-Max Depletion,Dragon
819,G-Max One Blow,Dark


In [13]:
# Converto i caratteri Maiuscoli in minuscoli
moves_is_type['Type']=moves_is_type['Type'].str.lower()
display(moves_is_type)

,Name,Type
0,Absorb,grass
1,Accelerock,rock
2,Acid,poison
3,Acid Armor,poison
4,Acid Spray,poison
...,...,...
816,G-Max Finale,fairy
817,G-Max Steelsurge,steel
818,G-Max Depletion,dragon
819,G-Max One Blow,dark


In [14]:
moves_is_type.to_csv('ponte_move_type_MOVES_IS_TYPE_import.csv')

In [ ]:
moves_is_type= '''
LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/rmarconzini/Dataset/main/ponte_move_type_MOVES_IS_TYPE_import.csv" AS row
MATCH (m:Moves {Name: row.Name}), (t:Type {Name: row.Type})
CREATE (m)-[:MOVES_IS_TYPE] -> (t)
'''

## Connessione al db ed esequzione delle query

In [8]:
# Connessione al Database
data_base_connection = GraphDatabase.driver(uri = "bolt://localhost:7687", auth=("neo4j", "pokemon"))
session = data_base_connection.session()    

In [ ]:
# Delete Relationship
delete_rel = '''
MATCH ()-[r: #Relationship]-() 
DELETE r
'''

In [ ]:
# ATTENZIONE QUESTA QUERY CANCELLA TUTTI I NODI !!!
delete_all = '''
MATCH (n) DELETE n
'''

### Import nodi

In [70]:
# POKEMON: Import vincoli e nodi
# session.run(pok_constraint)
# session.run(pok_import_nodes)

# MOVES: Import vincoli e nodi
# session.run(moves_constraint)
# session.run(moves_import_nodes)

# TYPES: Import vincoli e nodi
# session.run(types_constraint)
# session.run(types_import_nodes)

# ITEMS: Import vincoli e nodi
# session.run(items_constraint)
# session.run(items_import_nodes)

# ABILITIES: Import vincoli e nodi
# session.run(abilities_constraint)
# session.run(abilities_import_nodes)

# NATURES: Import vincoli e nodi
# session.run(natures_constraint)
# session.run(natures_import_nodes)

### Import relazioni

In [ ]:
# Import relazione: POKEMON - TYPE: IS_OF_TYPE
# session.run(is_of_type_rel)

# Import relazione: POKEMON - POKEMON: EVOLVES_INTO
# session.run(evolves_into_rel)

# Import relazione: POKEMON - MOVES: MAY_LEARN
# session.run(mey_learn_rel)

# Import relazione: GIGANTAMX - MAXMOVES: MAY_LEARN_GMAX
# session.run(may_learn_gmax_rel)

# Import relazione POKEMON - ITEM: USED_WITH_ITEM
# session.run(used_with_item_rel)

# Import relazione: POKEMON - ABILITY: USED_WITH_ABILITY
# session.run(used_with_ability)

# Import relazione: POKEMON - NATURE: COMMON_SPREAD
# session.run(common_spread_rel)

# Import relazione: POKEMON - MOVE: USED_WITH_MOVE
# session.run(used_with_move_rel)

# Import relazione: TYPE - TYPE: MOVE_EFFECTIVENESS_ON_POKEMON
# session.run(move_effectiveness_on_pokemon_rel)

# Import relazione: MOVES - TYPE: MOVES_IS_TYPE
# session.run(moves_is_type_rel)